In [2]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [3]:
week = 21 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

13 March 2023 ; 12:47:55


In [4]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [5]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,30,0,9.7,1.6,3.6,0.444,...,3.96,115.25,111.4,3.84,3.88,115.31,111.52,3.79,1,1
1,Bobby Portis,PF,27,MIL,56,18,26.1,5.8,11.5,0.502,...,3.96,115.25,111.4,3.84,3.88,115.31,111.52,3.79,0,1
5,Jae Crowder,SF,32,MIL,9,0,18.0,1.9,4.1,0.459,...,3.96,115.25,111.4,3.84,3.88,115.31,111.52,3.79,0,1
7,Joe Ingles,SF,35,MIL,34,0,22.9,2.4,5.6,0.429,...,3.96,115.25,111.4,3.84,3.88,115.31,111.52,3.79,0,1
9,Khris Middleton,SF,31,MIL,24,10,23.0,4.9,11.7,0.416,...,3.96,115.25,111.4,3.84,3.88,115.31,111.52,3.79,0,1


In [6]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [7]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,25.6,57,0,BOS,18.2,2.445614,6.761404,14.6,2,2.4,4.315789,0.302051
1,Bobby Portis,26.1,56,18,MIL,18.2,1.610714,6.589286,14.0,1,1.3,3.075000,0.141443
2,Malik Monk,22.5,63,0,SAC,16.7,0.911111,3.514286,13.7,2,1.7,2.342857,0.091610
3,Khris Middleton,23.0,24,10,MIL,16.2,0.683333,3.758333,13.8,1,-0.2,1.025000,0.060897
4,Immanuel Quickley,28.4,69,14,NYK,15.1,1.901449,6.179710,13.5,6,0.8,3.921739,0.059568


In [8]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)